<a href="https://colab.research.google.com/github/Swagaat/CE350/blob/main/Hw2_CE305_Swagat_neupane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def XOR(check_value, gen_poly):
    N = len(gen_poly)
    # If both bits are the same, the output is 0; if different, the output is 1
    for j in range(1, N):
        check_value[j] = '0' if check_value[j] == gen_poly[j] else '1'

def crc(data, gen_poly):
    N = len(gen_poly)
    data_length = len(data)
    check_value = list(data[:N])

    i = 0
    while i <= data_length + N - 1:
        if check_value[0] == '1':
            XOR(check_value, gen_poly)

        for j in range(N - 1):
            check_value[j] = check_value[j + 1]

        if i < data_length:
            check_value[-1] = data[i]
        else:
            check_value[-1] = '0'

        i += 1

    return ''.join(check_value)

def receiver(data, gen_poly):
    received_data = input("Enter the received data: ")

    print(f"Data received: {received_data}")

    # Cyclic Redundancy Check
    computed_check_value = crc(received_data, gen_poly)

    if '1' in computed_check_value:
        print("\nError detected\n")
    else:
        print("\nNo error detected\n")

def main():
    data = input("Enter data to be transmitted: ")
    gen_poly = input("Enter the Generating polynomial: ")

    data_length = len(data)
    N = len(gen_poly)

    # Append N-1 zeros to the data
    data += '0' * (N - 1)

    print("\n----------------------------------------")
    print(f"Data padded with n-1 zeros: {data}")
    print("\n----------------------------------------")

    computed_check_value = crc(data, gen_poly)

    print(f"CRC or Check value is: {computed_check_value}")

    # Append the computed check value to the data
    data = data[:data_length] + computed_check_value

    print("\n----------------------------------------")
    print(f"Final data to be sent: {data}")
    print("\n----------------------------------------")

    receiver(data, gen_poly)




if __name__ == "__main__":
    main()


Enter data to be transmitted: 1010
Enter the Generating polynomial: 100101

----------------------------------------
Data padded with n-1 zeros: 101000000

----------------------------------------
CRC or Check value is: 011100

----------------------------------------
Final data to be sent: 1010011100

----------------------------------------
Enter the received data: 1010011100
Data received: 1010011100

No error detected



In [2]:
def HamEncoding(msg):
    # Calculate the number of parity bits required
    m = len(msg)
    parity_bits = 0
    while 2 ** parity_bits < m + parity_bits + 1:
        parity_bits += 1

    # Create a list to hold the codeword with placeholders for parity bits
    codeword = [0] * (m + parity_bits)
    j = 0  # Index for the codeword
    k = 0  # Index for the message

    # Fill in the codeword with data bits and placeholders for parity bits
    for i in range(1, len(codeword) + 1):
        if i == 2 ** j:
            codeword[i - 1] = 0  # Initialize parity bits to 0
            j += 1
        else:
            codeword[i - 1] = int(msg[k])
            k += 1

    # Calculate and set the parity bits
    for j in range(parity_bits):
        mask = 2 ** j  # Mask to identify bits for this parity bit
        parity_bit = 0

        for i in range(1, len(codeword) + 1):
            if i & mask:  # Check if the bit at position i corresponds to this parity bit
                parity_bit ^= codeword[i - 1]

        codeword[mask - 1] = parity_bit  # Set the parity bit

    return ''.join(map(str, codeword))

def HamDecoding(rcv, k):
    codeword = [int(bit) for bit in rcv]

    # Calculate the number of parity bits from the received codeword length
    parity_bits = len(codeword).bit_length() - 1

    # Check and correct errors
    error_position = 0
    for j in range(parity_bits):
        mask = 2 ** j  # Mask to identify bits for this parity bit
        parity_bit = 0

        for i in range(1, len(codeword) + 1):
            if i & mask:  # Check if the bit at position i corresponds to this parity bit
                parity_bit ^= codeword[i - 1]

        if parity_bit != codeword[mask - 1]:
            error_position += mask

    if error_position == 0:
        return 'No error'
    else:
        codeword[error_position - 1] = 1 - codeword[error_position - 1]  # Correct the bit
        corrected_data = ''.join(map(str, codeword))
        return f'Error at Position {error_position}, and correct data: {corrected_data}'

# Example usage:
org_sig1 = '1101'
k1 = 3
encoded_sig1 = HamEncoding(org_sig1)
print(encoded_sig1)

received_sig1 = '1010101'
result1 = HamDecoding(received_sig1, k1)
print(result1)

org_sig2 = '1001011'
k2 = 4
encoded_sig2 = HamEncoding(org_sig2)
print(encoded_sig2)

received_sig2 = '10110010011'
result2 = HamDecoding(received_sig2, k2)
print(result2)

received_sig3 = '10110010011'
k3 = 4
result3 = HamDecoding(received_sig3, k3)
print(result3)
# Output: 'No error'

received_sig4 = '10110000011'
k4 = 4
result4 = HamDecoding(received_sig4, k4)
print(result4)

1010101
Error at Position 1, and correct data: 0010101
10110010011
Error at Position 5, and correct data: 10111010011
Error at Position 5, and correct data: 10111010011
Error at Position 2, and correct data: 11110000011
